In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,2021-07-16T17:00:00,ITA,19,Sicilia,38.115697,13.362357,144,23,167,4242,...,NaN,2.0,NaN,NaN,234661.0,0.0,2707132.0,2364537.0,ITG,ITG1
10685,2021-07-16T17:00:00,ITA,9,Toscana,43.769231,11.255889,60,16,76,1598,...,NaN,1.0,NaN,NaN,241988.0,3523.0,3951594.0,1357953.0,ITI,ITI1
10686,2021-07-16T17:00:00,ITA,10,Umbria,43.106758,12.388247,4,1,5,657,...,NaN,0.0,NaN,NaN,57049.0,0.0,993670.0,522501.0,ITI,ITI2
10687,2021-07-16T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,0,0,0,22,...,NaN,0.0,NaN,NaN,11027.0,681.0,102063.0,43221.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,2021-07-16T17:00:00,19,Sicilia,144,23,167,4242,4409,269,386,...,234661,5071669,1920876.0,2.0,234661.0,0.0,2707132.0,2364537.0,ITG,ITG1
10685,2021-07-16T17:00:00,9,Toscana,60,16,76,1598,1674,107,169,...,245511,5309547,2427278.0,1.0,241988.0,3523.0,3951594.0,1357953.0,ITI,ITI1
10686,2021-07-16T17:00:00,10,Umbria,4,1,5,657,662,45,56,...,57049,1516171,398530.0,0.0,57049.0,0.0,993670.0,522501.0,ITI,ITI2
10687,2021-07-16T17:00:00,2,Valle d'Aosta,0,0,0,22,22,3,5,...,11708,145284,68786.0,0.0,11027.0,681.0,102063.0,43221.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-07-16', '2021-07-15')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-07-16T17:00:00,1,Piemonte,48,3,51,869,920,82,115,...,363627,5249917,2045489.0,0.0,344833.0,18794.0,3133253.0,2116664.0,ITC,ITC1
1,2021-07-16T17:00:00,2,Valle d'Aosta,0,0,0,22,22,3,5,...,11708,145284,68786.0,0.0,11027.0,681.0,102063.0,43221.0,ITC,ITC2
2,2021-07-16T17:00:00,3,Lombardia,143,30,173,8080,8253,376,399,...,845091,12065399,4766855.0,2.0,790031.0,55060.0,9621393.0,2444006.0,ITC,ITC4
3,2021-07-16T17:00:00,5,Veneto,41,11,52,6045,6097,352,425,...,427747,9051554,1874840.0,1.0,415574.0,12173.0,5811809.0,3239745.0,ITH,ITH3
4,2021-07-16T17:00:00,6,Friuli Venezia Giulia,6,1,7,226,233,2,26,...,107214,2232644,698715.0,0.0,92711.0,14503.0,1846265.0,386379.0,ITH,ITH4
5,2021-07-16T17:00:00,7,Liguria,8,5,13,243,256,26,54,...,103755,1833076,708673.0,0.0,103755.0,0.0,1413230.0,419846.0,ITC,ITC3
6,2021-07-16T17:00:00,8,Emilia-Romagna,148,15,163,2195,2358,101,200,...,388272,6976623,1930574.0,1.0,387950.0,322.0,5107674.0,1868949.0,ITH,ITH5
7,2021-07-16T17:00:00,9,Toscana,60,16,76,1598,1674,107,169,...,245511,5309547,2427278.0,1.0,241988.0,3523.0,3951594.0,1357953.0,ITI,ITI1
8,2021-07-16T17:00:00,10,Umbria,4,1,5,657,662,45,56,...,57049,1516171,398530.0,0.0,57049.0,0.0,993670.0,522501.0,ITI,ITI2
9,2021-07-16T17:00:00,11,Marche,11,2,13,1406,1419,28,52,...,104199,1333388,778959.0,0.0,104199.0,0.0,1186466.0,146922.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-07-15T17:00:00,1,Piemonte,49,4,53,785,838,37,68,...,363512,5229285,2041199.0,1.0,344723.0,18789.0,3127606.0,2101679.0,ITC,ITC1
1,2021-07-15T17:00:00,2,Valle d'Aosta,0,0,0,19,19,0,0,...,11703,144928,68574.0,0.0,11023.0,680.0,101961.0,42967.0,ITC,ITC2
2,2021-07-15T17:00:00,3,Lombardia,128,29,157,7720,7877,-323,381,...,844692,12030688,4757739.0,0.0,789765.0,54927.0,9604761.0,2425927.0,ITC,ITC4
3,2021-07-15T17:00:00,5,Veneto,42,10,52,5693,5745,253,318,...,427322,9021475,1870696.0,2.0,415210.0,12112.0,5799066.0,3222409.0,ITH,ITH3
4,2021-07-15T17:00:00,6,Friuli Venezia Giulia,5,1,6,225,231,1,21,...,107188,2226593,697551.0,0.0,92690.0,14498.0,1842016.0,384577.0,ITH,ITH4
5,2021-07-15T17:00:00,7,Liguria,7,5,12,218,230,22,31,...,103701,1827676,706884.0,0.0,103701.0,0.0,1410392.0,417284.0,ITC,ITC3
6,2021-07-15T17:00:00,8,Emilia-Romagna,140,14,154,2103,2257,77,167,...,388073,6956769,1929200.0,1.0,387751.0,322.0,5100096.0,1856673.0,ITH,ITH5
7,2021-07-15T17:00:00,9,Toscana,62,15,77,1490,1567,86,173,...,245342,5298127,2422494.0,1.0,241824.0,3518.0,3945965.0,1352162.0,ITI,ITI1
8,2021-07-15T17:00:00,10,Umbria,7,1,8,609,617,-9,20,...,56993,1512006,398051.0,0.0,56993.0,0.0,992284.0,519722.0,ITI,ITI2
9,2021-07-15T17:00:00,11,Marche,9,2,11,1380,1391,17,40,...,104147,1330934,777728.0,0.0,104147.0,0.0,1184510.0,146424.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-07-16T17:00:00,1,Piemonte,48,3,51,869,920,82,115,...,5249917,2045489.0,0.0,344833.0,18794.0,3133253.0,2116664.0,ITC,ITC1,4.341375e+06
1,2021-07-16T17:00:00,2,Valle d'Aosta,0,0,0,22,22,3,5,...,145284,68786.0,0.0,11027.0,681.0,102063.0,43221.0,ITC,ITC2,1.255010e+05
2,2021-07-16T17:00:00,3,Lombardia,143,30,173,8080,8253,376,399,...,12065399,4766855.0,2.0,790031.0,55060.0,9621393.0,2444006.0,ITC,ITC4,1.010397e+07
3,2021-07-16T17:00:00,5,Veneto,41,11,52,6045,6097,352,425,...,9051554,1874840.0,1.0,415574.0,12173.0,5811809.0,3239745.0,ITH,ITH3,4.907704e+06
4,2021-07-16T17:00:00,6,Friuli Venezia Giulia,6,1,7,226,233,2,26,...,2232644,698715.0,0.0,92711.0,14503.0,1846265.0,386379.0,ITH,ITH4,1.211357e+06
5,2021-07-16T17:00:00,7,Liguria,8,5,13,243,256,26,54,...,1833076,708673.0,0.0,103755.0,0.0,1413230.0,419846.0,ITC,ITC3,1.543127e+06
6,2021-07-16T17:00:00,8,Emilia-Romagna,148,15,163,2195,2358,101,200,...,6976623,1930574.0,1.0,387950.0,322.0,5107674.0,1868949.0,ITH,ITH5,4.467118e+06
7,2021-07-16T17:00:00,9,Toscana,60,16,76,1598,1674,107,169,...,5309547,2427278.0,1.0,241988.0,3523.0,3951594.0,1357953.0,ITI,ITI1,3.722729e+06
8,2021-07-16T17:00:00,10,Umbria,4,1,5,657,662,45,56,...,1516171,398530.0,0.0,57049.0,0.0,993670.0,522501.0,ITI,ITI2,8.802850e+05
9,2021-07-16T17:00:00,11,Marche,11,2,13,1406,1419,28,52,...,1333388,778959.0,0.0,104199.0,0.0,1186466.0,146922.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-07-16T17:00:00,1,Piemonte,48,3,51,869,920,82,115,...,2116664.0,ITC,ITC1,4.341375e+06,Piemonte,48,5824,3,628,99
1,2021-07-16T17:00:00,2,Valle d'Aosta,0,0,0,22,22,3,5,...,43221.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,0,67,0,15,6
2,2021-07-16T17:00:00,3,Lombardia,143,30,173,8080,8253,376,399,...,2444006.0,ITC,ITC4,1.010397e+07,Lombardia,143,6369,30,1416,114
3,2021-07-16T17:00:00,5,Veneto,41,11,52,6045,6097,352,425,...,3239745.0,ITH,ITH3,4.907704e+06,Veneto,41,6000,11,1000,0
4,2021-07-16T17:00:00,6,Friuli Venezia Giulia,6,1,7,226,233,2,26,...,386379.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,6,1277,1,175,0
5,2021-07-16T17:00:00,7,Liguria,8,5,13,243,256,26,54,...,419846.0,ITC,ITC3,1.543127e+06,Liguria,8,1702,5,212,12
6,2021-07-16T17:00:00,8,Emilia-Romagna,148,15,163,2195,2358,101,200,...,1868949.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,148,6538,15,760,0
7,2021-07-16T17:00:00,9,Toscana,60,16,76,1598,1674,107,169,...,1357953.0,ITI,ITI1,3.722729e+06,Toscana,60,5033,16,775,37
8,2021-07-16T17:00:00,10,Umbria,4,1,5,657,662,45,56,...,522501.0,ITI,ITI2,8.802850e+05,Umbria,4,671,1,97,30
9,2021-07-16T17:00:00,11,Marche,11,2,13,1406,1419,28,52,...,146922.0,ITI,ITI3,1.518400e+06,Marche,11,951,2,209,57


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-07-16T17:00:00,12,Lazio,105,25,130,2515,2645,255,443,...,2.65,255,90,25980,3723.0,2,186,1.71,0.00755,5.94
1,2021-07-16T17:00:00,5,Veneto,41,11,52,6045,6097,352,425,...,1.10,352,107,30079,4144.0,1,72,1.41,0.00866,8.72
2,2021-07-16T17:00:00,3,Lombardia,143,30,173,8080,8253,376,399,...,2.12,360,18,34711,9116.0,2,21,1.15,0.00395,8.36
3,2021-07-16T17:00:00,19,Sicilia,144,23,167,4242,4409,269,386,...,3.54,263,33,9583,4611.0,0,117,4.03,0.00777,4.72
4,2021-07-16T17:00:00,15,Campania,171,12,183,6725,6908,0,204,...,2.09,24,-30,12807,2041.0,4,200,1.59,0.00353,7.38
5,2021-07-16T17:00:00,8,Emilia-Romagna,148,15,163,2195,2358,101,200,...,1.97,92,33,19854,1374.0,0,98,1.01,0.00448,8.69
6,2021-07-16T17:00:00,9,Toscana,60,16,76,1598,1674,107,169,...,2.06,108,-4,11420,4784.0,1,61,1.48,0.00454,6.59
7,2021-07-16T17:00:00,20,Sardegna,43,3,46,1353,1399,135,140,...,1.39,134,28,4160,2493.0,0,5,3.37,0.00859,3.56
8,2021-07-16T17:00:00,1,Piemonte,48,3,51,869,920,82,115,...,0.48,84,47,20632,4290.0,0,33,0.56,0.00265,8.38
9,2021-07-16T17:00:00,18,Calabria,47,3,50,1955,2005,19,77,...,1.99,19,45,1937,1724.0,0,58,3.98,0.00400,3.61


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Sicilia,23,627,650,3.54
1,Lazio,25,918,943,2.65
2,Liguria,5,207,212,2.36
3,Lombardia,30,1386,1416,2.12
4,Campania,12,563,575,2.09
5,Toscana,16,759,775,2.06
6,Calabria,3,148,151,1.99
7,Emilia-Romagna,15,745,760,1.97
8,Puglia,9,453,462,1.95
9,P.A. Trento,1,59,60,1.67


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))